In [10]:
import subprocess
import os
import time

from tqdm import tqdm

# 使用FFmpeg将视频转换为图片，每秒一帧

In [11]:
def extract_key_frames_ffmpeg(video_path, output_base_folder, frame_rate=1):
    """使用 FFmpeg 提取视频中的关键帧并将它们保存到以视频命名的文件夹中。

    Args:
        video_path (str): 视频文件的路径。
        output_base_folder (str): 存储所有关键帧文件夹的基础路径。
        frame_rate (int, optional): 提取关键帧的频率。默认每秒提取一帧。
    """
    video_name = os.path.basename(video_path).split('.')[0]
    output_folder = os.path.join(output_base_folder, video_name)
    
    # 创建视频对应的文件夹（如果不存在的话）
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_filename = os.path.join(output_folder, "frame_%d.jpg")
    cmd = f"./ffmpeg -i {video_path} -vf fps={frame_rate} {output_filename}"
    subprocess.run(cmd, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)


def process_videos_with_ffmpeg(folder_path, output_folder, frame_rate=1):
    """使用 FFmpeg 处理文件夹中的所有视频，并显示进度条。

    Args:
        folder_path (str): 包含视频文件的文件夹路径。
        output_folder (str): 存储关键帧的文件夹路径。
        frame_rate (int, optional): 提取关键帧的频率。默认每秒提取一帧。
    """
    videos = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.mp4')]

    with tqdm(total=len(videos), desc="Processing Videos") as pbar:
        for video in videos:
            extract_key_frames_ffmpeg(video, output_folder, frame_rate)
            pbar.update(1)

In [12]:
# 训练集和测试集的路径
train_videos_path = '../Data_Q3/train_video'
test_videos_path = '../Data_Q3/test_video'

# 输出文件夹
output_folder_train = '../Processed_Data_Q3/train_key_frames'
output_folder_test = '../Processed_Data_Q3/test_key_frames'

# 处理训练集和测试集中的视频
process_videos_with_ffmpeg(train_videos_path, output_folder_train)
process_videos_with_ffmpeg(test_videos_path, output_folder_test)


Processing Videos: 100%|██████████| 562/562 [01:01<00:00,  9.07it/s]


# Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os